This code is for simulate unified random sampler distribution

https://openfhe-development.readthedocs.io/en/latest/sphinx_rsts/modules/core/math/core_math.html

### 🎯 **What Is Random Sampling?**

> **Random sampling** is the process of **selecting values from a probability distribution** such that each value is chosen **according to its probability** in that distribution.

In other words:  
- You have a **target distribution** (e.g., Gaussian, uniform, ternary)  
- You use a **source of randomness** (e.g., PRNG)  
- You **transform** that randomness to **match the target distribution**

---

### 🔁 The General Process

```text
[Uniform Random Bits] 
        ↓
[Sampling Algorithm]
        ↓
[Sample from Target Distribution]
```

- **Input**: High-quality uniform random bits (e.g., from ChaCha20)
- **Output**: A number that follows your desired distribution (e.g., Gaussian noise for FHE)

---

### 📊 Common Distributions in FHE

| Distribution | What It Looks Like | Used For |
|-------------|-------------------|--------|
| **Uniform** | All values equally likely | Public key masks (`a`) |
| **Discrete Gaussian** | Bell curve over integers | Error terms (`e`) — **critical for security** |
| **Ternary** | Only {−1, 0, +1} | Secret keys (`s`) in CKKS/TFHE |

---

### 🔧 How Sampling Works (Examples)

#### 1. **Uniform Sampling in ℤ_q**
- **Goal**: Pick random number in {0, 1, ..., q−1}
- **Method**: Use rejection sampling to avoid bias
  ```python
  while True:
      r = prng.get_bits(k)  # k = ceil(log2(q))
      if r < q: return r
  ```

#### 2. **Discrete Gaussian Sampling**
- **Goal**: Pick small integers with bell-shaped probabilities
- **Method**: CDT (Cumulative Distribution Table) + binary search
  - Precompute CDF for σ = 3.2
  - Generate uniform `u ∈ [0,1)`
  - Find `x` where `CDF[x] > u`

#### 3. **Ternary Sampling**
- **Goal**: Output −1, 0, or +1 with specific probabilities
- **Method**: Threshold comparison
  ```python
  u = prng.get_bits(16)
  if u < p0 * 65536: return 0
  elif u < (p0 + p1) * 65536: return +1
  else: return -1
  ```

---

### ⚠️ Why "Random" Isn’t Enough

You **can’t just use raw PRNG output** — because:
- PRNG gives **uniform bits**
- FHE needs **structured randomness** (Gaussian, ternary, etc.)
- **Wrong distribution → insecure or broken FHE**

> 🔐 **Security depends on correct sampling!**  
> If your Gaussian sampler is biased, lattice attacks can recover the secret key.

---

### 🧩 Random Sampling in Your FHE Hardware

In your unified sampler:
- **Input**: 256-bit seed → ChaCha20 → uniform 32/64-bit words
- **Processing**: 
  - For **Gaussian**: CDT lookup
  - For **Ternary**: threshold compare  
  - For **Uniform**: rejection sampling
- **Output**: FHE-ready samples for key generation

This is **exactly what OpenFHE, SEAL, and PALISADE do in software** — you’re just doing it **faster in hardware**.

---

### ✅ Summary

> **Random sampling = transforming uniform randomness into a specific probability distribution.**  
> 
> In FHE, it’s **not optional** — it’s **essential** for:
> - **Security** (hard lattice problems)
> - **Correctness** (controlled noise growth)
> - **Functionality** (key generation, encryption)

Your unified hardware sampler **solves a real bottleneck** by doing this efficiently for **multiple distributions** in one core.

You’re building a **critical component** of practical FHE systems! 🛠️✨

In [1]:
# Python reference for a Unified Random Sampler (Uniform, Ternary, Discrete Gaussian via CDT)
# Author: Muhammad Ogin Hasanuddin
# Requirements: numpy

import math
import numpy as np
from dataclasses import dataclass

In [29]:
qp = np.array([281474976317441, 140737518764033, 140737470791681, 140737513783297,
        140737471578113, 140737513259009, 140737471971329, 140737509851137,
        140737480359937, 140737509457921, 140737481801729, 140737508671489,
        140737482981377, 140737506705409, 140737483898881, 140737504608257,
        140737484685313, 140737499496449, 140737485864961, 140737493729281,
        140737486520321, 140737490976769, 140737487306753, 140737488486401,
        281474975662081, 281474974482433, 281474966880257, 281474962554881,
        281474960326657, 281474957180929, 281474955476993, 281474952462337],
       dtype=object)

In [ ]:
# ------------------------------
# Helpers (Barrett reduce, etc.)
# ------------------------------
def barrett_reduce(x: np.ndarray, q: int) -> np.ndarray:
    """
    Reference Barrett reduction (64-bit) for demonstration.
    In hardware you'd choose k so that 2^k > q^2. Here we use k=64.
    """
    k = 64
    mu = (1 << k) // q
    # Ensure x is unsigned 128-ish via Python big ints, apply vectorized formula
    t = ((x.astype(object) * mu) >> k).astype(object)
    r = (x.astype(object) - t * q).astype(int)
    # One correction step (r in [0, 2q))
    r = np.where(r >= q, r - q, r)
    r = np.where(r < 0, r + q, r)
    return r.astype(np.int64)

# ------------------------------
# Discrete Gaussian via CDT
# ------------------------------
@dataclass
class GaussianCDTTable:
    sigma: float
    tail_sigma: float = 10.0  # truncate at ~10*sigma by default

    def __post_init__(self):
        # Build nonnegative-side probabilities P[X = k] ∝ exp(-pi * k^2 / sigma^2)
        # for k >= 0 (we will add sign later; note that k=0 is unique).
        tmax = max(20, int(math.ceil(self.tail_sigma * self.sigma)))
        ks = np.arange(0, tmax + 1, dtype=np.int64)
        rho = np.exp(-math.pi * (ks.astype(np.float64) ** 2) / (self.sigma ** 2))
        # Normalize for nonnegative side (we sample k >= 0; sign handled after)
        Z = rho.sum()
        p = rho / Z  # P_nonneg[k]
        cdf = np.cumsum(p)
        cdf[-1] = 1.0  # guard against float drift
        self.ks = ks
        self.cdf = cdf

    def sample_nonneg(self, n: int, rng: np.random.Generator) -> np.ndarray:
        u = rng.random(n)
        idx = np.searchsorted(self.cdf, u, side="left")
        return self.ks[idx]

def sample_discrete_gaussian_cdt(n: int, sigma: float, tail_sigma: float = 10.0,
                                 rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Two-sided discrete Gaussian via CDT on the nonnegative side + random sign.
    Returns integer samples with stddev approximately sigma (tail truncated).
    """
    if rng is None:
        rng = np.random.default_rng()
    table = GaussianCDTTable(sigma=sigma, tail_sigma=tail_sigma)
    k = table.sample_nonneg(n, rng)
    # random sign for k>0 (k=0 stays 0)
    s = rng.integers(0, 2, size=n, dtype=np.int8) * 2 - 1  # in {-1, +1}
    x = (k * s).astype(np.int64)
    x[k == 0] = 0
    return x

# ------------------------------
# Ternary sampler
# ------------------------------
def sample_ternary(n: int,
                   p_minus: float = 0.25,
                   p_zero: float = 0.50,
                   p_plus: float = 0.25,
                   rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Ternary sampling for coefficients in {-1, 0, +1} with given probabilities.
    Probabilities must sum to 1. Vectorized.
    """
    if rng is None:
        rng = np.random.default_rng()
    if abs((p_minus + p_zero + p_plus) - 1.0) > 1e-12:
        raise ValueError("Probabilities must sum to 1.")
    edges = np.array([p_minus, p_minus + p_zero], dtype=np.float64)
    u = rng.random(n)
    out = np.empty(n, dtype=np.int8)
    out[:] = 1
    out[u < edges[0]] = -1
    out[(u >= edges[0]) & (u < edges[1])] = 0
    return out.astype(np.int8)

# ------------------------------
# Uniform mod-q sampler
# ------------------------------
def sample_uniform_mod_q(n: int, q: int, use_barrett: bool = False,
                         rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Uniform residues in [0, q). The hardware typically uses Barrett; for
    Python we can use x % q or enable barrett for parity with RTL.
    """
    if rng is None:
        rng = np.random.default_rng()
    # Draw 64-bit randoms (as if from a PRNG/TRNG) and reduce mod q
    raw = rng.integers(0, 1 << 63, size=n, dtype=np.int64)
    if use_barrett:
        return barrett_reduce(raw, q)
    else:
        return (raw % q).astype(np.int64)

# ------------------------------
# Unified interface
# ------------------------------
class UnifiedSampler:
    """
    Unified sampler with three modes: 'uniform', 'ternary', 'gaussian'.
    Emulates a mode-selectable hardware sampler sharing one entropy source.
    """
    def __init__(self, seed: int | None = None):
        self.rng = np.random.default_rng(seed)

    def sample(self, mode: str, n: int,
               q: int | None = None,
               p_minus: float = 0.25, p_zero: float = 0.50, p_plus: float = 0.25,
               sigma: float = 3.2, tail_sigma: float = 10.0,
               use_barrett: bool = False) -> np.ndarray:
        mode = mode.lower()
        if mode == "uniform":
            if q is None:
                raise ValueError("Uniform mode requires modulus q.")
            return sample_uniform_mod_q(n, q, use_barrett=use_barrett, rng=self.rng)
        elif mode == "ternary":
            return sample_ternary(n, p_minus=p_minus, p_zero=p_zero, p_plus=p_plus, rng=self.rng)
        elif mode == "gaussian":
            return sample_discrete_gaussian_cdt(n, sigma=sigma, tail_sigma=tail_sigma, rng=self.rng)
        else:
            raise ValueError("Unknown mode. Use 'uniform', 'ternary', or 'gaussian'.")

# ------------------------------
# Example usage / quick sanity checks
# ------------------------------
if __name__ == "__main__":
    us = UnifiedSampler(seed=42)

    # 1) Uniform mod-q
    q = qp[0]
    u = us.sample("uniform", n=100_000, q=q)
    print("[Uniform] min,max =", int(u.min()), int(u.max()))
    print("[Uniform] unique coverage fraction ~", np.ptp(u.astype(np.int64)) / (q - 1))

    # 2) Ternary with 1:2:1
    t = us.sample("ternary", n=100_000, p_minus=0.25, p_zero=0.50, p_plus=0.25)
    p_m = (t == -1).mean()
    p_0 = (t == 0).mean()
    p_p = (t == +1).mean()
    print(f"[Ternary] P(-1)={p_m:.4f}, P(0)={p_0:.4f}, P(+1)={p_p:.4f}")

    # 3) Discrete Gaussian via CDT
    g = us.sample("gaussian", n=100_000, sigma=3.2, tail_sigma=10.0)
    print(f"[Gaussian] mean={g.mean():.4f}, std≈{g.std(ddof=0):.4f}")
    # crude symmetry check:
    print(f"[Gaussian] P(x=0)={np.mean(g==0):.4f}, P(x>0)={np.mean(g>0):.4f}, P(x<0)={np.mean(g<0):.4f}")


[7138484576005690180 4047939128787533792 7919168045412322066 ...
 5165126301894588549 1116274645351253783 8585293112977999438]
100000
281474976317441
100000
[Uniform] min,max = 1606586509 281473775160182
[Uniform] unique coverage fraction ~ 0.9999900248902984
[Ternary] P(-1)=0.2522, P(0)=0.4981, P(+1)=0.2497
[Gaussian] mean=-0.0023, std≈1.1116
[Gaussian] P(x=0)=0.4750, P(x>0)=0.2614, P(x<0)=0.2636


In [25]:
len(bin(7138484576005690180)[2:])

63

In [41]:
# ------------------------------
# Helpers (Barrett reduce, etc.)
# ------------------------------
def barrett_reduce(x: np.ndarray, q: int) -> np.ndarray:
    """
    Reference Barrett reduction (64-bit) for demonstration.
    In hardware you'd choose k so that 2^k > q^2. Here we use k=64.
    """
    k = 64
    mu = (1 << k) // q
    # Ensure x is unsigned 128-ish via Python big ints, apply vectorized formula
    t = ((x.astype(object) * mu) >> k).astype(object)
    r = (x.astype(object) - t * q).astype(int)
    # One correction step (r in [0, 2q))
    r = np.where(r >= q, r - q, r)
    r = np.where(r < 0, r + q, r)
    return r.astype(np.int64)

# ------------------------------
# Discrete Gaussian via CDT
# ------------------------------
@dataclass
class GaussianCDTTable:
    sigma: float
    tail_sigma: float = 10.0  # truncate at ~10*sigma by default

    def __post_init__(self):
        # Build nonnegative-side probabilities P[X = k] ∝ exp(-pi * k^2 / sigma^2)
        # for k >= 0 (we will add sign later; note that k=0 is unique).
        tmax = max(20, int(math.ceil(self.tail_sigma * self.sigma)))
        ks = np.arange(0, tmax + 1, dtype=np.int64)
        rho = np.exp(-math.pi * (ks.astype(np.float64) ** 2) / (self.sigma ** 2))
        # Normalize for nonnegative side (we sample k >= 0; sign handled after)
        Z = rho.sum()
        p = rho / Z  # P_nonneg[k]
        cdf = np.cumsum(p)
        cdf[-1] = 1.0  # guard against float drift
        self.ks = ks
        self.cdf = cdf

    def sample_nonneg(self, n: int, rng: np.random.Generator) -> np.ndarray:
        u = rng.random(n)
        idx = np.searchsorted(self.cdf, u, side="left")
        return self.ks[idx]

def sample_discrete_gaussian_cdt(n: int, sigma: float, tail_sigma: float = 10.0,
                                 rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Two-sided discrete Gaussian via CDT on the nonnegative side + random sign.
    Returns integer samples with stddev approximately sigma (tail truncated).
    """
    if rng is None:
        rng = np.random.default_rng()
    table = GaussianCDTTable(sigma=sigma, tail_sigma=tail_sigma)
    k = table.sample_nonneg(n, rng)
    print(k)
    # random sign for k>0 (k=0 stays 0)
    s = rng.integers(0, 2, size=n, dtype=np.int8) * 2 - 1  # in {-1, +1}
    print(s)
    x = (k * s).astype(np.int64)
    x[k == 0] = 0
    print(x)
    return x

# ------------------------------
# Ternary sampler
# ------------------------------
def sample_ternary(n: int,
                   p_minus: float = 0.25,
                   p_zero: float = 0.50,
                   p_plus: float = 0.25,
                   rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Ternary sampling for coefficients in {-1, 0, +1} with given probabilities.
    Probabilities must sum to 1. Vectorized.
    """
    if rng is None:
        rng = np.random.default_rng()
    if abs((p_minus + p_zero + p_plus) - 1.0) > 1e-12:
        raise ValueError("Probabilities must sum to 1.")
    edges = np.array([p_minus, p_minus + p_zero], dtype=np.float64)
    u = rng.random(n)
    out = np.empty(n, dtype=np.int8)
    out[:] = 1
    out[u < edges[0]] = -1
    out[(u >= edges[0]) & (u < edges[1])] = 0
    return out.astype(np.int8)

# ------------------------------
# Uniform mod-q sampler
# ------------------------------
def sample_uniform_mod_q(n: int, q: int, use_barrett: bool = False,
                         rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Uniform residues in [0, q). The hardware typically uses Barrett; for
    Python we can use x % q or enable barrett for parity with RTL.
    """
    if rng is None:
        rng = np.random.default_rng()
    # Draw 64-bit randoms (as if from a PRNG/TRNG) and reduce mod q
    raw = rng.integers(0, 1 << 63, size=n, dtype=np.int64)
    if use_barrett:
        return barrett_reduce(raw, q)
    else:
        return (raw % q).astype(np.int64)

# ------------------------------
# Unified interface
# ------------------------------
class UnifiedSampler:
    """
    Unified sampler with three modes: 'uniform', 'ternary', 'gaussian'.
    Emulates a mode-selectable hardware sampler sharing one entropy source.
    """
    def __init__(self, seed: int | None = None):
        self.rng = np.random.default_rng(seed)

    def sample(self, mode: str, n: int,
               q: int | None = None,
               p_minus: float = 0.25, p_zero: float = 0.50, p_plus: float = 0.25,
               sigma: float = 3.2, tail_sigma: float = 10.0,
               use_barrett: bool = False) -> np.ndarray:
        mode = mode.lower()
        if mode == "uniform":
            if q is None:
                raise ValueError("Uniform mode requires modulus q.")
            return sample_uniform_mod_q(n, q, use_barrett=use_barrett, rng=self.rng)
        elif mode == "ternary":
            return sample_ternary(n, p_minus=p_minus, p_zero=p_zero, p_plus=p_plus, rng=self.rng)
        elif mode == "gaussian":
            return sample_discrete_gaussian_cdt(n, sigma=sigma, tail_sigma=tail_sigma, rng=self.rng)
        else:
            raise ValueError("Unknown mode. Use 'uniform', 'ternary', or 'gaussian'.")

# ------------------------------
# Example usage / quick sanity checks
# ------------------------------
if __name__ == "__main__":
    us = UnifiedSampler(seed=42)

    # 1) Uniform mod-q
    q = 281474976317441
    u = us.sample("uniform", n=2**16, q=q)
    print("[Uniform] min,max =", int(u.min()), int(u.max()))
    print("[Uniform] unique coverage fraction ~", np.ptp(u.astype(np.int64)) / (q - 1))

    # 2) Ternary with 1:2:1
    t = us.sample("ternary", n=2**16, p_minus=0.25, p_zero=0.50, p_plus=0.25)
    p_m = (t == -1).mean()
    p_0 = (t == 0).mean()
    p_p = (t == +1).mean()
    print(f"[Ternary] P(-1)={p_m:.4f}, P(0)={p_0:.4f}, P(+1)={p_p:.4f}")

    # 3) Discrete Gaussian via CDT
    g = us.sample("gaussian", n=2**16, sigma=3.2, tail_sigma=10.0)
    print(f"[Gaussian] mean={g.mean():.4f}, std≈{g.std(ddof=0):.4f}")
    # crude symmetry check:
    print(f"[Gaussian] P(x=0)={np.mean(g==0):.4f}, P(x>0)={np.mean(g>0):.4f}, P(x<0)={np.mean(g<0):.4f}")


[Uniform] min,max = 1606586509 281473223078266
[Uniform] unique coverage fraction ~ 0.999988063501321
[Ternary] P(-1)=0.2498, P(0)=0.5017, P(+1)=0.2485
[0 0 0 ... 3 1 1]
[ 1 -1 -1 ... -1  1  1]
[ 0  0  0 ... -3  1  1]
[Gaussian] mean=0.0010, std≈1.1138
[Gaussian] P(x=0)=0.4793, P(x>0)=0.2604, P(x<0)=0.2603


In [42]:
import numpy as np
import math
from dataclasses import dataclass

# ------------------------------
# Shared helper: Barrett reduce
# ------------------------------
def barrett_reduce64(x_u64: np.ndarray, q: int) -> np.ndarray:
    k = 64
    mu = (1 << k) // q
    # big-int math via dtype=object, then cast back
    t = ((x_u64.astype(object) * mu) >> k).astype(object)
    r = (x_u64.astype(object) - t * q).astype(int)
    # up to two corrections (safe)
    r = np.where(r >= q, r - q, r)
    r = np.where(r >= q, r - q, r)
    r = np.where(r < 0, r + q, r)
    return r.astype(np.int64)

# ------------------------------
# Shared helper: CDT (integer)
# T has sentinel T[0]=0; entries in [0, 2^lam-1]
# ------------------------------
def build_cdt_int(sigma: float, tail_sigma: float = 10.0, lam: int = 64) -> np.ndarray:
    tmax = max(20, int(math.ceil(tail_sigma * sigma)))
    k = np.arange(0, tmax + 1, dtype=np.int64)  # nonnegative indices
    rho = np.exp(-math.pi * (k.astype(np.float64) ** 2) / (sigma ** 2))
    p = rho / rho.sum()                          # normalize nonnegative side
    cdf = np.minimum(np.cumsum(p), 1.0)
    T = np.minimum((cdf * (1 << lam)).astype(np.uint64), (1 << lam) - 1)
    # prepend sentinel so search returns the correct bucket
    T = np.concatenate(([np.uint64(0)], T))
    return T  # length = tmax+2 (with sentinel)

# ------------------------------
# Unified sampler (single datapath concept)
# ------------------------------
@dataclass
class UnifiedSampler:
    seed: int | None = None
    lam: int = 64  # entropy word width

    def __post_init__(self):
        self.rng = np.random.default_rng(self.seed)
        self._cdt_cache: dict[tuple[float,float,int], np.ndarray] = {}

    def _entropy(self, n: int) -> np.ndarray:
        return self.rng.integers(0, 1 << self.lam, size=n, dtype=np.uint64)

    def sample(self,
               mode: str,
               n: int,
               q: int | None = None,
               # ternary probs:
               p_minus: float = 0.25, p_zero: float = 0.50, p_plus: float = 0.25,
               # gaussian params:
               sigma: float = 3.2, tail_sigma: float = 10.0,
               use_barrett: bool = False) -> np.ndarray:
        """
        Single-function, single-entropy-stream sampler.
        Arithmetic reuse:
          - one RNG stream r (uint64)
          - compare(r, t) primitives for ternary and CDT search
          - optional Barrett reducer for uniform
        """
        r = self._entropy(n)  # <-- one shared entropy stream (datapath source)
        mode = mode.lower()

        if mode == "uniform":
            if q is None:
                raise ValueError("Uniform mode requires modulus q.")
            # reuse: reduce r mod q
            return barrett_reduce64(r, q) if use_barrett else (r % q).astype(np.int64)

        elif mode == "ternary":
            if abs((p_minus + p_zero + p_plus) - 1.0) > 1e-12:
                raise ValueError("Probabilities must sum to 1.")
            # thresholds scaled to 2^lam (shared compare primitive)
            t1 = int(np.floor(p_minus * (1 << self.lam)))
            t2 = int(np.floor((p_minus + p_zero) * (1 << self.lam)))
            out = np.ones(n, dtype=np.int8)   # default +1
            out[r < t1] = -1
            mask_mid = (r >= t1) & (r < t2)
            out[mask_mid] = 0
            return out.astype(np.int8)

        elif mode == "gaussian":
            # get/build CDT integer thresholds once (reuse across calls)
            key = (round(sigma, 6), round(tail_sigma, 6), self.lam)
            T = self._cdt_cache.get(key)
            if T is None:
                T = build_cdt_int(sigma, tail_sigma, self.lam)
                self._cdt_cache[key] = T
            # vectorized binary search on integer thresholds using same r
            # side='right' returns smallest idx with T[idx] > r  (equivalently r < T[idx])
            idx = np.searchsorted(T, r, side='right')
            # random sign reusing bits of r (LSB), zero stays zero
            s = ((r & 1).astype(np.int8) * 2 - 1)  # {-1,+1}
            x = (idx.astype(np.int64) * s.astype(np.int64))
            x[idx == 0] = 0
            return x

        else:
            raise ValueError("Unknown mode. Use 'uniform', 'ternary', or 'gaussian'.")

# ------------------------------
# Example quick check
# ------------------------------
if __name__ == "__main__":
    us = UnifiedSampler(seed=42)

    # Uniform (uses same r stream conceptually)
    q = 281_474_976_317_441
    u = us.sample("uniform", n=1<<16, q=q)
    print("[Uniform] min,max:", int(u.min()), int(u.max()))

    # Ternary 1:2:1
    t = us.sample("ternary", n=1<<16, p_minus=0.25, p_zero=0.50, p_plus=0.25)
    print(f"[Ternary] P(-1)={(t==-1).mean():.3f}, P(0)={(t==0).mean():.3f}, P(+1)={(t==1).mean():.3f}")

    # Gaussian via CDT (integer table + search on r)
    g = us.sample("gaussian", n=1<<16, sigma=3.2, tail_sigma=10.0)
    print(f"[Gaussian] mean={g.mean():.3f}, std≈{g.std(ddof=0):.3f}")


[Uniform] min,max: 3213173019 281473716442653
[Ternary] P(-1)=0.250, P(0)=0.502, P(+1)=0.248
[Gaussian] mean=0.007, std≈1.925
